# Analysing the redundancies of User Stories - Annotations but no US Text

In [8]:
import locale
import os, json
from itertools import combinations

import pandas as pd
import tiktoken
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter as utils_get_column_letter

from prompt_structure.helper_prompt_composition import PromptHelperBuilder
from prompt_structure.prompt_builder import PromptBuilder

from support_functions.excel_helper import save_to_excel, formatter_ignored_items
from support_functions.load_data import load_datasets_with_annotations as loading
from support_functions.request_handler import process_user_stories, process_user_stories_parallel
from support_functions.data_transformations import convert_annotation_dataset
from support_functions.time_recorder import TimeRecorder
from support_functions.json_validator import validation, chat_gpt_schema_with_annotations

In [9]:
locale.setlocale(locale.LC_ALL, 'de_DE') 
MODEL_VERSION_NAME = os.getenv('MODEL_VERSION')
THREADING = bool(int(os.getenv('THREADING')))
print(f"Threading is used: {THREADING}")
print(MODEL_VERSION_NAME)

Threading is used: True
gpt-3.5-turbo


In [10]:
time_recorders: dict = {}
time_recorder: TimeRecorder = None

### Load all User Stories from the different packages

We use here just the annotations (formatted):
- Triggers
- Targets
- Contains

We do not need:
- POS

Ignored Files as they differer from the common structure [Ref](https://github.com/ace-design/nlp-stories/tree/main?tab=readme-ov-file#note):
- g02-federal-funding
- g13-planningpoker
- g17-cask
- g27-culrepo

Ignored Files as they were used as trainings data
- g05
- g12

Data sets to analyse:
 - 'g03', 'g04', 'g08', 'g10', 'g11', 'g14', 'g16', 'g18', 'g19', 'g21', 'g22', 'g23', 'g24', 'g25', 'g26', 'g28'

Used Data:
- "nlp" --> "nlp_outputs" --> "individual_backlog" --> "nlp_outputs_original" --> "pos_baseline"

Highlighting words with #

In [11]:
not_processed_datasets: dict[str, list] = loading()
print(not_processed_datasets["g02"][0])

{'PID': '#G02#', 'Text': '#G02# As a Data user, I want to have the 12-19-2017 deletions processed.', 'Persona': ['Data user'], 'Action': {'Primary Action': ['have'], 'Secondary Action': ['processed']}, 'Entity': {'Primary Entity': ['12-19-2017 deletions'], 'Secondary Entity': ['']}, 'Benefit': '', 'Triggers': [['Data user', 'have']], 'Targets': [['processed', '12-19-2017 deletions'], ['have', '12-19-2017 deletions']], 'Contains': [], 'Persona POS': {'Persona POS tag': [['NOUN', 'NOUN']], 'Persona POS text': [['Data', 'user']]}, 'Action POS': {'Primary Action POS': {'Primary Action POS tag': [['VERB']], 'Primary Action POS text': [['have']]}, 'Secondary Action POS': {'Secondary Action POS tag': [['VERB']], 'Secondary Action POS text': [['processed']]}}, 'Entity POS': {'Primary Entity POS': {'Primary Entity POS tag': [['NUM', 'NOUN']], 'Primary Entity POS text': [['12-19-2017', 'deletions']]}, 'Secondary Entity POS': {'Secondary Entity POS tag': [[]], 'Secondary Entity POS text': [[]]}}}

### Converting Data to used suited format

In [12]:
datasets, ignored_items = convert_annotation_dataset(not_processed_datasets)

In [13]:
datasets.keys()

dict_keys(['g02', 'g03', 'g04', 'g05', 'g08', 'g10', 'g11', 'g12', 'g13', 'g14', 'g16', 'g17', 'g18', 'g19', 'g21', 'g22', 'g23', 'g24', 'g25', 'g26', 'g27', 'g28'])

In [14]:
datasets["g05"]

[{'PID': '#G05#',
  'USID': '373',
  'Text': 'As a Data Publishing User, I want to be able to edit a dataset I have published, So that I can correct or enhance existing data.',
  'Main Part': 'As a Data Publishing User, I want to be able to edit a dataset I have published',
  'Benefit': 'I can correct or enhance existing data',
  'Triggers': {'Main Part': [['Data Publishing User', 'edit']], 'Benefit': []},
  'Targets': {'Main Part': [['edit', 'dataset I have published']],
   'Benefit': [['correct', 'existing data'], ['enhance', 'existing data']]},
  'Contains': {'Main Part': [], 'Benefit': []}},
 {'PID': '#G05#',
  'USID': '374',
  'Text': 'As a Data Publishing User, I want to be able to edit the model of data I have already imported, So that I can fix bugs or make enhancements in the API built for my data.',
  'Main Part': 'As a Data Publishing User, I want to be able to edit the model of data I have already imported',
  'Benefit': 'I can fix bugs or make enhancements in the API built

In [15]:
ignored_items["#G02#"][0]

'PID: #G02#; Text: As a UI designer, I want to redesign the Resources page, so that it matches the new Broker design styles.; Label Type: Contain'

In [16]:
df_ignored_items: pd.DataFrame = None
if True:            
    cols: list[str] =  ["PID", "Text", "Label Type"]
    df_ignored_items = pd.DataFrame(columns=cols)
    current_ignored_data: dict = {}
    curremt_pid: int = 0
    current_text: str = "" 
    for key, set in ignored_items.items():
        for item in set:
            parts = item.split('; ')
            for part in parts:
                key, value = part.split(': ', 1)
                current_ignored_data[key] = value
            df_ignored_items = pd.concat([df_ignored_items, pd.DataFrame([current_ignored_data])], ignore_index = True)               
            df_ignored_items.reset_index()
    save_to_excel(df_ignored_items, formatter_ignored_items, "Incomplete US labelling", os.getenv("OUTPUT_EXCEL_NAME_WITH_ANNOTATIONS"))
print(f"Count of ignored Elements: {df_ignored_items.shape[0]}")
df_ignored_items.head()

Count of ignored Elements: 195


,PID,Text,Label Type
0,#G02#,"As a UI designer, I want to redesign the Resou...",Contain
1,#G02#,"As a UI designer, I want to track the issues t...",Target
2,#G02#,"As a UI designer, I want to track the issues t...",Target
3,#G02#,"As a UI designer, I want to track the issues t...",Target
4,#G02#,"As an agency user, I want to submit my data el...",Contain


### Datatransformation to panda dataframes and numpy arrarys. 

In [17]:
def transform_2_dataframe(data: list):
    rows = []
    for entry in data:
        row = {
            "PID": entry["PID"],
            "USID": entry["USID"],
            "Text": entry["Text"],
            "Main Part": entry["Main Part"],
            "Benefit": entry["Benefit"],
            "Triggers": entry["Triggers"],
            "Targets": entry["Targets"],
            "Contains": entry["Contains"],
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [18]:
# n(n-1) / 2
def transform_pairwise(df: pd.DataFrame):
    rows = []
    for i, j in combinations(range(df.shape[0]), 2):
        row = {
            "First PID": df.iloc[i, 0],
            "First USID": df.iloc[i, 1],
            "First Text": df.iloc[i, 2],
            "First Main Part": df.iloc[i, 3],
            "First Benefit": df.iloc[i, 4],
            "First Triggers": df.iloc[i, 5],
            "First Targets": df.iloc[i, 6],
            "First Contains": df.iloc[i, 7],
            "Second PID": df.iloc[j, 0],
            "Second USID": df.iloc[j, 1],
            "Second Text": df.iloc[j, 2],
            "Second Main Part": df.iloc[j, 3],
            "Second Benefit": df.iloc[j, 4],
            "Second Triggers": df.iloc[j, 5],
            "Second Targets": df.iloc[j, 6],
            "Second Contains": df.iloc[j, 7]
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [19]:
def convert_dataframe_to_json(pair: pd.Series) -> tuple[dict, dict]:
    pair = pair.to_dict()
    us_one: dict = {
        "USID": pair["First USID"],
        "Triggers": pair["First Triggers"],
        "Targets": pair["First Targets"],
        "Contains": pair["First Contains"]
    }
    
    us_two: dict = {
        "USID": pair["Second USID"],
        "Triggers": pair["Second Triggers"],
        "Targets": pair["Second Targets"],
        "Contains": pair["Second Contains"]
    }

    json_one: dict = json.loads(json.dumps(us_one))
    json_two: dict = json.loads(json.dumps(us_two))
    
    return json_one, json_two

In [20]:
def template_request_two_user_stories(
    current_message: list[dict],
    idx: int,
    pairs: pd.DataFrame,
) -> None:
    row = pairs.iloc[idx]
    json_us_one: dict = None
    json_us_two: dict = None
    json_us_one, json_us_two = convert_dataframe_to_json(row)
    request: dict = PromptHelperBuilder.get_instance().parsing_to_pair_requests(json_us_one, json_us_two)
    current_message.append(request)

In [21]:
def sort_threaded_results(to_sort: dict) -> None:
    new_list: list = None
    for _ in to_sort.values():
        new_list = sorted(_, key=lambda x: (int(x['relatedStories'][0]), int(x['relatedStories'][1])))
        _.clear()
        _ += new_list

In [22]:
def json_validator(json_data: dict) -> tuple[bool, str]:
    return validation(json_data, chat_gpt_schema_with_annotations)

### Prepaire Prompting

In [23]:
# Init PromptBuilder
builder: PromptBuilder = PromptBuilder.get_instance()

# Collection of messages
message: list[dict] = []

In [24]:
# Actor Role
message.append(builder.get_actor_role())
message.append(builder.get_system_simulation_actor_role())

In [25]:
# User Story Definition
message.append(builder.get_user_story_definition())
message.append(builder.get_system_simulation_user_story_definition())

In [26]:
# Redundancy Definition for User Story pair
message.append(builder.get_redundancy_full_partial_definition())
message.append(builder.get_system_simulation_redundancy_full_partial_definition())

In [27]:
# Defining the JSON output
message.append(builder.get_json_defintion())
message.append(builder.get_system_simulation_redundancy_full_partial_definition())

In [28]:
# Providing Examples
message.append(builder.get_input_output_examples(["USID", "Triggers", "Targets", "Contains"]))
message.append(builder.get_system_simulation_example_consideration())

In [29]:
if True:
    message_text: str = ""
    for key in message:
        message_text += key["content"]
        print(key["content"])
    enc = tiktoken.get_encoding('cl100k_base')
    token_size = enc.encode(message_text)
    print("-" * 3 + "Token" + "-" * 3)
    print(token_size)
    print("-" * 3 + "Sum of Tokens" + "-" * 3)
    # The output is not correct as the result from the online pages differs https://platform.openai.com/tokenizer
    print("The total token sum is: " + locale.format_string("%d", sum(token_size), grouping=True))
    

Act as a Requirements Engineer focused on identifying redundancies. Please review pairs of two User Stories and pinpoint any unnecessary duplications that obscure clarity or add no distinct value.
I will analyse redundancies in the main parts and benefits of two User Stories. Each story might include multiple redundancies. The main part typically describes the desired functionality by the persona, while the benefit details the positive outcomes from the functionality.
Please, analyse redundancies in the main part and benefit of a pair of two given User Stories which are entered as JSON objects. Note that a User Story pair may include multiple redundancies in the main part as well as the benefit. The redundancies of the main part and benefit are disjoint sets. Hence, a main part can be redundant while a benefit is not and vice versa. However, in some cases the main part and the benefit can be at the same time redundant, but they do not depend on each other and therefore they are indepen

TypeError: 'method' object is not subscriptable

## Data processing for: G03

## Save total Speeds

In [ ]:
SHEET_NAME = 'Time Consup. Anlys.S.'

In [ ]:
base_path: str = os.getcwd()
path_to_file = os.path.join(base_path, "results")
path_to_file = os.path.join(path_to_file, "redundancy-model-" + MODEL_VERSION_NAME)

count_runs_per_data_set: dict[str, int] = {}
for key in time_recorders.keys():
        idx = 0
        _ =  f"{idx:02d}_{key}.json"
        while os.path.exists(os.path.join(path_to_file, f"{idx:02d}_{key}.json")):
                idx += 1
        count_runs_per_data_set[key] = idx

entries: list[tuple[int,str,str, int, float, float, float]] = []

entry: tuple[int,str,str, int, float, float, float] = None
for key, value in time_recorders.items():
        entry = (key, count_runs_per_data_set[key], MODEL_VERSION_NAME, str(THREADING).lower(), 
                 value.nanoseconds, value.milliseconds, value.seconds, value.minutes)
        entries.append(entry)

columns = ['Dataset', 'Run Count', 'Model Version', 'Threading Enabled', 
           'Nanoseconds', 'Milliseconds', 'Seconds', 'Minutes']

_file_path: str = os.path.join(base_path, os.getenv('OUTPUT_EXCEL_NAME_WITH_ANNOTATIONS_MODIFIED'))
check = os.path.exists(_file_path)

old_time_consuption_data: pd.DataFrame = None
if check:
        try:
                old_time_consuption_data = pd.read_excel(_file_path, SHEET_NAME)
        except ValueError:
                check = False
           
time_consumption_data: pd.DataFrame = pd.DataFrame(entries, columns=columns)

if check and not old_time_consuption_data.empty:
        # Checking if in the excel is already the data. Case: this code is executed twice for the same data
        old_time_consuption_data = old_time_consuption_data.dropna()
        for idx in range(len(old_time_consuption_data)):
                condition = time_consumption_data[
                        (time_consumption_data['Dataset'] == old_time_consuption_data.iat[idx, 0]) & 
                        (time_consumption_data['Run Count'] == old_time_consuption_data.iat[idx, 1])
                ].index
                time_consumption_data = time_consumption_data.drop(condition).reset_index(drop=True)
        time_consumption_data = pd.concat([old_time_consuption_data, time_consumption_data]).reset_index(drop=True)

In [ ]:
def formatter_time(wb: Workbook, sheet_name: str): 
    ws = wb[sheet_name]
    header_font = Font(size=14, bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
    
    ADDITIONAL_LENGTH: int = 0
    ADJUSTED_WIDTH: int = 0
    MAX_LEN: int = 0
    for col in ws.iter_cols(min_row=1, max_row=1):
        for cell in col:
            MAX_LEN = len(str(cell.value))
            ADDITIONAL_LENGTH = (MAX_LEN + 2)
            ADJUSTED_WIDTH = 0
            ADJUSTED_WIDTH =  ADDITIONAL_LENGTH * 1.5
            ws.column_dimensions[utils_get_column_letter(cell.column)].width = ADJUSTED_WIDTH
            
    alignment = Alignment(vertical='center', horizontal='left')
    for row in ws.iter_rows():
        for cell in row:
            cell.alignment = alignment

    num_columns = ws.max_column
    header_range = f"A1:{utils_get_column_letter(num_columns)}1"
    ws.auto_filter.ref = header_range
    ws.freeze_panes = ws['A2']

    wrap_alignment = Alignment(wrap_text=True, vertical='top', horizontal='left')
    for row in ws.iter_rows(min_row=2):
        for cell in row:
            cell.alignment = wrap_alignment

In [ ]:
save_to_excel(time_consumption_data, formatter_time, SHEET_NAME, os.getenv("OUTPUT_EXCEL_NAME_WITH_ANNOTATIONS"))